In [1]:
# imports
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# batch_size
batch_size = 8

# datasets
trainset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In C:\Users\ChristianBernecker\AppData\Local\Programs\Python\Python36\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\ChristianBernecker\AppData\Local\Programs\Python\Python36\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\ChristianBernecker\AppData\Local\Programs\Python\Python36\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\ChristianBernecker\AppData\Local\Programs\Python\Python36\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was

26422272it [00:04, 5302933.67it/s]                              


Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw


  0%|          | 0/29515 [00:00<?, ?it/s]

29696it [00:00, 1246987.02it/s]          
  0%|          | 0/4422102 [00:00<?, ?it/s]

...

4422656it [00:00, 4903053.82it/s]                             


Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



6144it [00:00, 1501037.03it/s]          

Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw




C:\Users\ChristianBernecker\AppData\Roaming\Python\Python36\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [3]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

def createConfusionMatrix(loader):
    y_pred = [] # save predction
    y_true = [] # save ground truth

    # iterate over data
    for inputs, labels in loader:
        output = net(inputs)  # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output)  # save prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels)  # save ground truth

    # constant for classes
    classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

    # Build confusion matrix
    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) * 10, index=[i for i in classes],
                         columns=[i for i in classes])
    plt.figure(figsize=(12, 7))    
    return sn.heatmap(df_cm, annot=True).get_figure()

In [4]:
from torch.utils.tensorboard import SummaryWriter

running_loss = 0.0
accuracy = 0
epochs = 15

# tensorboard file
writer = SummaryWriter('runs/fashion_mnist')

for epoch in range(epochs):  # loop over the dataset multiple times
    print('Epoch-{0} lr: {1}'.format(epoch + 1, optimizer.param_groups[0]['lr']))
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data # get the inputs; data is a list of [inputs, labels]
        optimizer.zero_grad() # zero the parameter gradients
        
        outputs = net(inputs) # forward
        loss = criterion(outputs, labels) # calculate loss
        loss.backward() # backward loss
        optimizer.step() # optimize gradients

        running_loss += loss.item() # save loss
        _, preds = torch.max(outputs, 1) # save prediction
        accuracy += torch.sum(preds == labels.data) # save accuracy
        
        if i % 1000 == 999:    # every 1000 mini-batches...           
            steps = epoch * len(trainloader) + i # calculate steps 
            batch = i*batch_size # calculate batch 
            print("Training loss {:.3} Accuracy {:.3} Steps: {}".format(running_loss / batch, accuracy/batch, steps))
            
            # Save accuracy and loss to Tensorboard
            writer.add_scalar('Training loss by steps', running_loss / batch, steps)
            writer.add_scalar('Training accuracy by steps', accuracy / batch, steps)
            
            
    print("Accuracy: {}/{} ({:.3} %) Loss: {:.3}".format(accuracy, len(trainloader), 100. * accuracy / len(trainloader.dataset), running_loss / len(trainloader.dataset)))
    
    # Save confusion matrix to Tensorboard
    writer.add_figure("Confusion matrix", createConfusionMatrix(trainloader), epoch)
    
    running_loss = 0.0
    accuracy = 0
    
print('Finished Training')

Epoch-1 lr: 0.001


C:\Users\ChristianBernecker\AppData\Local\Programs\Python\Python36\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
C:\Users\ChristianBernecker\AppData\Local\Programs\Python\Python36\lib\site-packages\torch\autograd\__init__.py:149: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:115.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


Training loss 0.19 Accuracy 0.454 Steps: 999
Training loss 0.141 Accuracy 0.584 Steps: 1999
Training loss 0.12 Accuracy 0.642 Steps: 2999
Training loss 0.108 Accuracy 0.676 Steps: 3999
Training loss 0.1 Accuracy 0.7 Steps: 4999
Training loss 0.0941 Accuracy 0.718 Steps: 5999
Training loss 0.0894 Accuracy 0.732 Steps: 6999
Accuracy: 44276/7500 (73.8 %) Loss: 0.0874
Epoch-2 lr: 0.001
Training loss 0.057 Accuracy 0.833 Steps: 8499
Training loss 0.0563 Accuracy 0.834 Steps: 9499
Training loss 0.0551 Accuracy 0.837 Steps: 10499
Training loss 0.0544 Accuracy 0.84 Steps: 11499
Training loss 0.0536 Accuracy 0.843 Steps: 12499
Training loss 0.0532 Accuracy 0.844 Steps: 13499
Training loss 0.0524 Accuracy 0.846 Steps: 14499
Accuracy: 50841/7500 (84.7 %) Loss: 0.052
Epoch-3 lr: 0.001
Training loss 0.0469 Accuracy 0.864 Steps: 15999
Training loss 0.0471 Accuracy 0.863 Steps: 16999
Training loss 0.0466 Accuracy 0.864 Steps: 17999
Training loss 0.0459 Accuracy 0.866 Steps: 18999
Training loss 0.0457